In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.packages.all_observable_grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

Generating from folder  boxes
boxes:box_1159 :  <class 'spatial_scene_grammars_examples.packages.all_observable_grammar.boxes:box_1159'>
boxes:box_0968 :  <class 'spatial_scene_grammars_examples.packages.all_observable_grammar.boxes:box_0968'>
boxes:box_1265 :  <class 'spatial_scene_grammars_examples.packages.all_observable_grammar.boxes:box_1265'>
boxes:box_0010 :  <class 'spatial_scene_grammars_examples.packages.all_observable_grammar.boxes:box_0010'>
boxes:box_0765 :  <class 'spatial_scene_grammars_examples.packages.all_observable_grammar.boxes:box_0765'>


In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Meshcat url:  http://127.0.0.1:7000/static/


In [3]:
# Draw a random sample from the grammar with its initial params and visualize it.
#torch.random.manual_seed(5)

torch.random.manual_seed(46)
grammar = SpatialSceneGrammar(
    root_node_type = BoxGroup,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.]))
)

pre_projection_tree = grammar.sample_tree(detach=True)
nlp_projected_tree = project_tree_to_feasibility(deepcopy(pre_projection_tree), jitter_q=0.01)
tree = project_tree_to_feasibility(deepcopy(pre_projection_tree), do_forward_sim=True, zmq_url=vis.window.zmq_url)

vis["sample"].delete()
draw_scene_tree_contents_meshcat(pre_projection_tree, zmq_url=vis.window.zmq_url, prefix="sample/pre_projected/contents")
draw_scene_tree_structure_meshcat(pre_projection_tree, zmq_url=vis.window.zmq_url, prefix="sample/pre_projected/structure")

draw_scene_tree_contents_meshcat(nlp_projected_tree, zmq_url=vis.window.zmq_url, prefix="sample/nlp_projected/contents")
draw_scene_tree_structure_meshcat(nlp_projected_tree, zmq_url=vis.window.zmq_url, prefix="sample/nlp_projected/structure")

draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/sim_projected/contents")
draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/sim_projected/structure")

[2021-09-29 23:34:44.780] [console] [warning] FindResource ignoring DRAKE_RESOURCE_ROOT='/home/gizatt/drake' because it does not contain a 'drake' subdirectory.


In [4]:
#samples = do_fixed_structure_hmc_with_constraint_penalties(
#    grammar, tree, num_samples=100, subsample_step=2,
#    with_nonpenetration=True, zmq_url=vis.window.zmq_url
#)

In [5]:
#for k, tree in enumerate(samples):
#    draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="samples/contents/%d/structure" % k)
#    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="samples/structure/%d/structure" % k)

# Simulated langevin

In [ ]:
simulation_tree = deepcopy(tree)
builder, mbp, scene_graph, node_to_free_body_ids_map, body_id_to_node_map = compile_scene_tree_to_mbp_and_sg(simulation_tree)
mbp.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph,
    zmq_url=vis.window.zmq_url, prefix="simulation")

langevin_source = builder.AddSystem(
    StochasticLangevinForceSource(mbp, simulation_tree, node_to_free_body_ids_map, body_id_to_node_map)
)
builder.Connect(mbp.get_state_output_port(),
                langevin_source.get_input_port(0))
builder.Connect(langevin_source.get_output_port(0),
                mbp.get_applied_spatial_force_input_port())

diagram = builder.Build()
diag_context = diagram.CreateDefaultContext()
sim = Simulator(diagram)
sim.set_target_realtime_rate(1.)
visualizer.start_recording()
sim.AdvanceTo(5.)
visualizer.stop_recording()
visualizer.publish_recording(play=True, repetitions=1)